In [1]:
#import necessary libraries for regression analysis
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.model_selection import train_test_split

### Custom Functions ###

In [2]:
def create_ols (df, target = 'price'):
    
    """Inputs: 
    df = data as data frame
    target : str = name of target variable
    
    
    Retruns: an sm.OLS model
    Prints out the summary table
    ------------------------
    """ 
    
    
    X = df.drop(target, axis=1)
    model = ols(formula='price~' + '+'.join(X.columns), data=df).fit()
    print(model.summary())
    return model

################


def test_homoskedasticity(model):
    
    """Input: 
    model: an sm.OLS model  
    
    Retrun: 
    True if 
    False  
    Prints out P-value and F-statistic
    ------------------------
    Uses the Breusch–Pagan test. If p-value is below 0.05, then 
    the null hypothesis of homoskedasticity is rejected and 
    heteroskedasticity is assumed.
    """ 

    _, pval, _, f_pval = diag.het_breuschpagan(model.resid, model.model.exog)
    
    print('-'*50)
    print('The P-value is: {:.2}'.format(pval))
    print('The F-statistic is {:.2}'.format(f_pval))
    print('-'*50)
    
    if pval >= 0.05:
        return True
    
    return False

################

def test_resid_distribution(model):
    """Input --> model: an sm.OLS model  
    
    Retruns: nothing
    Displays a Q-Q-Plot
    Prints out the Mean of the Residuals.
    ------------------------
    This function check for normality of the residuals with a Q-Q-Plot 
    and verifies by displaying the mean of the residuals. The closer the mean is 
    to 0, the more normal the distribution of the residuals. 
    """ 
    import pylab
    sm.qqplot(model.resid, line = 's')
    pylab.show()
    
    mean_residuals = sum(model.resid)/len(model.resid)

    print('The mean of the residuals is: {:.2}. The closer to 0, the better.'.format(mean_residuals))

################

def test_features_vif(df, target="",):
    """Input:
    df = data as data frame
    target : str = name of target variable
    
    Retruns: a list of features with a high variance inflation factor
    Displays a data frame with the VIFs for each feature
    
    ------------------------
    """ 
    
    X = df.drop(target, axis=1)
    vif = pd.DataFrame()
    vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif["features"] = X.columns
    display(vif.round(1))
    
    corr_features = list(vif[vif["VIF Factor"] > 30]['features'])
    return corr_features

###########

def get_low_pval_features(model):
    """Input -- > model - an OLS model
    
    Retruns: a list of features with p-value < 0.05 
    in the summary table
    
    ------------------------
    """ 
    
    summary = model.summary()
    ptable = summary.tables[1]
    df_p = pd.DataFrame(ptable.data)
    df_p.drop(0, inplace=True)
    df_p[4] = df_p[4].map(lambda x: float(x))
    relevant_list = list(df_p[df_p[4] < .05][0][1:])
    
    return relevant_list


In [3]:
df_preprocessed = pd.read_csv('Project2datacleaned', index_col=0)
df_preprocessed.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15,renovated,grade_binned,floors_binned
0,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,3,7,1180,0.0,1955,0.0,98178,1340,5650,0,1,0
1,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,1690,7639,0,1,1
2,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,0.0,98028,2720,8062,0,0,0
3,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,1360,5000,0,1,0
4,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,1800,7503,0,1,0


### Train-Test-Split

In [4]:
train, test = train_test_split(df_preprocessed, test_size=0.2, random_state=42)

In [5]:
print(len(train), len(test))

13449 3363


In [6]:
#Build baseline model
X = train.drop('price', axis=1)
formula = 'price~' + '+'.join(X.columns)

In [7]:
baseline = ols(formula=formula, data=train).fit()

In [8]:
baseline.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.416
Model:                            OLS   Adj. R-squared:                  0.416
Method:                 Least Squares   F-statistic:                     504.5
Date:                Wed, 18 Nov 2020   Prob (F-statistic):               0.00
Time:                        20:58:23   Log-Likelihood:            -1.7467e+05
No. Observations:               13449   AIC:                         3.494e+05
Df Residuals:                   13429   BIC:                         3.495e+05
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -1.534e+07   1.92e+06     -8.000      0.000   -1.91e+07   -1.16e+07
bedrooms       -1.41e+04   1501.694     -9.390      0.000    -1.7e+04   -1.12e+04
bathrooms      2.847e+04   2408.386     11.823      0.000    2.38e+04    3.32e+04
sqft_living      65.2765     15.668      4.166      0.000      34.566      95.987
sqft_lot          0.1148      0.042      2.767      0.006       0.033       0.196
floors         2.651e+04   8536.129      3.106      0.002    9778.296    4.32e+04
waterfront     6.759e+04   2.41e+04      2.806      0.005    2.04e+04    1.15e+05
view           5757.4951   1889.674      3.047      0.002    2053.468    9461.522
condition      1.141e+04   1566.490      7.284      0.000    8340.435    1.45e+04
grade          6.008e+04   1832.624     32.783      0.000    5.65e+04    6.37e+04
sqft_above      -42.8587     15.634     -2.741      0.006     -73.503     -12.214
sqft_basement    -3.2543     15.566     -0.209      0.834     -33.767      27.258
yr_built      -1812.6037     54.436    -33.298      0.000   -1919.305   -1705.902
yr_renovated    -12.4559      3.848     -3.237      0.001     -19.999      -4.913
zipcode         189.8992     19.250      9.865      0.000     152.167     227.631
sqft_living15    50.9599      2.757     18.483      0.000      45.556      56.364
sqft_lot15        0.0070      0.056      0.125      0.901      -0.103       0.117
renovated      1.743e+04   7240.405      2.407      0.016    3238.266    3.16e+04
grade_binned   2.082e+04   3667.620      5.677      0.000    1.36e+04     2.8e+04
floors_binned  1.935e+04   7426.571      2.606      0.009    4795.579    3.39e+04
==============================================================================
Omnibus:                      161.891   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              131.237
Skew:                           0.170   Prob(JB):                     3.18e-29
Kurtosis:                       2.655   Cond. No.                     2.10e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.1e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
baseline_test = create_ols(train,target='price')

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.416
Model:                            OLS   Adj. R-squared:                  0.416
Method:                 Least Squares   F-statistic:                     504.5
Date:                Wed, 18 Nov 2020   Prob (F-statistic):               0.00
Time:                        20:58:23   Log-Likelihood:            -1.7467e+05
No. Observations:               13449   AIC:                         3.494e+05
Df Residuals:                   13429   BIC:                         3.495e+05
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -1.534e+07   1.92e+06     -8.000

In [10]:
test_homoskedasticity(baseline_test)

--------------------------------------------------
The P-value is: 1.8e-50
The F-statistic is 4.5e-51
--------------------------------------------------


False

In [11]:
test_resid_distribution(baseline_test)

<Figure size 640x480 with 1 Axes>

The mean of the residuals is: 8.5e-08. The closer to 0, the better.


In [12]:
check_vif(baseline_test)

NameError: name 'check_vif' is not defined

In [13]:
test_features_vif(train,'price')

,VIF Factor,features
0,29.5,bedrooms
1,29.2,bathrooms
2,1073.6,sqft_living
3,2.6,sqft_lot
4,203.7,floors
5,1.1,waterfront
6,1.2,view
7,34.4,condition
8,217.7,grade
9,821.1,sqft_above


['sqft_living',
 'floors',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'zipcode',
 'sqft_living15',
 'floors_binned']

In [14]:
get_low_pval_features(baseline_test)

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'sqft_living15',
 'renovated',
 'grade_binned',
 'floors_binned']

### Model 1: Log Transform Variables

In [ ]:
#Adding dummies to categorical variables
floors_dummies = pd.get_dummies(df['floors_binned'], prefix='floors_binned', drop_first=True)
view_dummies = pd.get_dummies(df['view'], prefix='view', drop_first=True)
grade_dummies = pd.get_dummies(df['grade_binned'], prefix='grade_binned', drop_first=True)
condition_dummies = pd.get_dummies(df['condition'], prefix='condition', drop_first=True)

df_preprocessed = df.drop(['floors_binned', 'view', 'grade_binned', 'condition'], axis=1)
df_preprocessed = pd.concat([df_preprocessed, floors_dummies, view_dummies, grade_dummies, condition_dummies], axis=1)
df_preprocessed.info()

In [ ]:
#Rename columns that contain '.' so that they will run properly through our regression model later
df.columns = df.columns.str.replace('.','_')

In [ ]:
#Should we be removing floors and grade? The binned variables?